In [2]:
# install Keras 

In [4]:
import io

tweets = []
responses = []

with io.open("/Users/hannes/Downloads/twitter_corpus/test.txt", "rb") as twitter_data:
    for i, text in enumerate(twitter_data.readlines()):
        text = text.decode('ascii', 'ignore').strip().replace('\n', '')  # this will remove emojis
        if not i%2:  # line 1, 3, 5, ...
            tweets.append(text)
        else:
            responses.append(text)

In [5]:
# Check
i = 3
print("Tweet: {} \nReply: {}".format(tweets[i], responses[i]))
dataset = tweets + responses

Tweet: psych is being taken off of netflix tomorrow and i have 8 episodes left and i work tonight kill me 
Reply: before you go to work open each episode up in its own tab


In [6]:
index = 0
chars = list(set(''.join(dataset)))
max_len = max([len(i) for i in dataset])
print(sorted(chars))
print("Longest tweet found has {} characters".format(max_len))

[' ', "'", '-', '.', '8', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']
Longest tweet found has 132 characters


In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


In [12]:
import numpy as np

def convert_data(words, max_len, chars, char_indices):
    num_words = len(words)
    X = np.zeros((num_words, max_len, len(chars)), dtype=np.bool)
    for i in range(num_words):
        word = words[i]
        for j, c in enumerate(word):
            X[i, j, char_indices[c]] = 1
    return X

In [13]:
X = convert_data(tweets, max_len, chars, char_indices)
Y = convert_data(responses, max_len, chars, char_indices)

In [16]:
from keras.layers import Dense, Dropout, TimeDistributed, Bidirectional, Input, Activation
from keras.models import Sequential
from keras.layers.recurrent import SimpleRNN, GRU, LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(None, len(chars)), return_sequences=False))
model.add(Dropout(0.2))
model.add(RepeatVector(max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, batch_size=1, epochs=30)

ImportError: No module named 'keras'

In [ ]:
n_hidden

In [ ]:
'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be inverted, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits inverted:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits inverted:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits inverted:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits inverted:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent
import numpy as np
from six.moves import range
from keras.engine.training import _slice_arrays

class CharacterTable(object):
    '''
    Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    '''
    def __init__(self, chars, maxlen):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.maxlen = maxlen

    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.chars)))
        for i, c in enumerate(C):
            X[i, self.char_indices[c]] = 1
        return X

    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in X)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset
TRAINING_SIZE = 5000
DIGITS = 3
INVERT = True
# Try replacing GRU, or SimpleRNN
RNN = recurrent.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1
MAXLEN = DIGITS + 1 + DIGITS

chars = '0123456789+ '
ctable = CharacterTable(chars, MAXLEN)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that X+Y == Y+X (hence the sorting)
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
X = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, maxlen=DIGITS + 1)

# Shuffle (X, y) in unison as the later parts of X will almost all be larger digits
indices = np.arange(len(y))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over
split_at = len(X) - len(X) / 10
(X_train, X_val) = (_slice_arrays(X, 0, split_at), _slice_arrays(X, split_at))
(y_train, y_val) = (y[:split_at], y[split_at:])

print(X_train.shape)
print(y_train.shape)

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE
# note: in a situation where your input sequences have a variable length,
# use input_shape=(None, nb_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# For the decoder's input, we repeat the encoded input for each time step
model.add(RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# For each of step of the output sequence, decide which character should be chosen
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model each generation and show predictions against the validation dataset
for iteration in range(1, 2):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X_train, y_train, batch_size=BATCH_SIZE, nb_epoch=1,
              validation_data=(X_val, y_val))
    ###
    # Select 10 samples from the validation set at random so we can visualize errors
    for i in range(10):
        ind = np.random.randint(0, len(X_val))
        rowX, rowy = X_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowX, verbose=0)
        q = ctable.decode(rowX[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if INVERT else q)
        print('T', correct)
        print('X', guess)
        print(colors.ok + '☑' + colors.close if correct == guess else colors.fail + '☒' + colors.close, guess)
        print('---')


In [ ]:
X_train[3]

In [ ]:
from keras.models import Sequential
import numpy as np
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers import TimeDistributed, Activation

n_in_out = 1
n_hidden = 100
n_samples = 2297
n_timesteps = 400

model = Sequential()
# `return_sequences` controls whether to copy the input automatically
model.add(GRU( n_hidden, input_dim = n_in_out, return_sequences=True))
model.add(TimeDistributed(n_in_out))
model.compile(loss='mse', optimizer='rmsprop')

X = np.random.random((n_samples, n_timesteps, n_in))
Y = np.random.random((n_samples, n_timesteps, n_out))

# learning the hidden states from source sentences

Xp = model._predict(X)
print(Xp.shape)
print(Y.shape)

model.fit(X, Y, nb_epoch=10)

In [ ]:
??TimeDistributed

In [ ]:
# from keras.preprocessing.text import Tokenizer,base_filter
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,Embedding
from keras.layers import LSTM


# def shift(seq, n):
#     n = n % len(seq)
#     return seq[n:] + seq[:n]

# txt="abcdefghijklmn"*100

# tk = Tokenizer(nb_words=2000, filters=base_filter(), lower=True, split=" ")
# tk.fit_on_texts(txt)
# x = tk.texts_to_sequences(txt)
# #shifing to left
# y = shift(x,1)

X_train = np.asarray([map(lambda x: ord(x), i) for i in X])
Y_train = np.asarray([map(lambda y: ord(y), i) for i in Y])

# max_features=max_len
X_train = pad_sequences(X_train, maxlen=max_len)
Y_train = pad_sequences(Y_train, maxlen=max_len)

#lstm model
model = Sequential()
model.add(Embedding(max_features, len(max_features), input_length=max_len, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(max_len))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.fit(X, Y, batch_size=200, nb_epoch=10)

In [ ]:
import itertools
max_features = set(itertools.chain(*X_train))

In [ ]:
len(max_features)


In [ ]:
max_features

In [ ]:
from keras.layers import Dense, Dropout, TimeDistributed, Bidirectional, Input, Activation
from keras.models import Sequential
import numpy as np
from keras.layers.recurrent import SimpleRNN, GRU, LSTM

def convert_data(words, max_len, chars, char_indices):
    num_words = len(words)
    X = np.zeros((num_words, max_len, len(chars)), dtype=np.bool)
    for i in xrange(num_words):
        word = words[i]
        for j, c in enumerate(word):
            X[i, j, char_indices[c]] = 1
    return X


def decode(matrix, indices_char):
    rv = []
    for v in matrix:
        _m = v.max(keepdims=True)
        if _m:
            _v = (v == v.max(keepdims=True)).astype(int)
            rv.extend(''.join(indices_char[i] for i, x in enumerate(_v) if x))
    return rv

with open('missp.dat') as file:
    words = map(lambda x: x.replace('\n', ''), file.readlines())

X = []
Y = []

cor_word = ''
for w in words:
    if w.startswith('$'):
        cor_word = w.replace('$', '')
    else:
        Y.append(list(cor_word))
        X.append(list(w))
assert len(X) == len(Y)
print("Detected {} training pairs".format(len(X)))

index = 0
print("Misspelled: {}, Correctly Spelled: {}".format(X[index], Y[index]))
chars = list(set(''.join(words))).remove('$')
max_len = max([len(i) for i in X+Y])
print("Longest Word found has {} characters".format(max_len))

incorrect_words = [''.join(i).replace('_', ' ') for i in X[:100]]
correct_words = [''.join(i).replace('_', ' ') for i in Y[:100]]

CHARS = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .'-")
chars = CHARS
chars = sorted(set(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = convert_data(incorrect_words, max_len, chars, char_indices)
Y = convert_data(correct_words, max_len, chars, char_indices)

model = Sequential()
model.add(LSTM(128, input_shape=(None, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(None, len(chars)), return_sequences=False))
model.add(Dropout(0.2))
model.add(RepeatVector(max_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, batch_size=1, epochs=30)

In [ ]:
a = model.predict(X[1].reshape(1, 18, 56))  # correct spelling

In [ ]:
print("predicted: \t", "".join(decode(a[0], indices_char)))
print("Output: \t", "".join(decode(Y[1], indices_char)))
print("Input: \t\t", "".join(decode(X[1], indices_char)))

In [ ]:
len(incorrect_words)
X.shape

In [ ]:

def decode(matrix, indices_char):
    rv = []
    for v in matrix:
        _m = v.max(keepdims=True)
        if _m:
            _v = (v == v.max(keepdims=True)).astype(int)
            rv.extend(''.join(indices_char[i] for i, x in enumerate(_v) if x))
    return rv
decode(a[0], indices_char)

In [ ]:
X[0]